In [ ]:
import json
from collections import defaultdict
from pathlib import Path

import geopandas
import pandas

In [ ]:
with open("../config.json", "r") as fh:
    config = json.load(fh)
base_path = Path(config["base_path"])
base_path

## Polders, restoration sites

In [ ]:
DISTANCE_WITHIN_M = (
    5_000  # distance for selection of polder-to-restoration-site relations (in metres)
)

In [ ]:
protected_polders = geopandas.read_file(
    base_path / "infrastructure" / "protectedPolders" / "Polders_selected.shp",
    engine="pyogrio",
)

protected_polders_9678 = protected_polders.to_crs(epsg=9678)
protected_polders_4326 = protected_polders.to_crs(epsg=4326)

In [ ]:
len(protected_polders.polder_ID.unique()), len(protected_polders)

In [ ]:
restoration_sites = geopandas.read_file(
    base_path
    / "nature-ecosystems"
    / "Potential Mangrove Restoration Sites"
    / "potential_sites_epsg9678.gpkg",
    engine="pyogrio",
).explode(index_parts=False)

In [ ]:
restoration_sites["potential_site_id"] = range(len(restoration_sites))

In [ ]:
restoration_sites.tail(2)

In [ ]:
restoration_sites_buf = restoration_sites.copy()
restoration_sites_buf.geometry = restoration_sites.buffer(DISTANCE_WITHIN_M)

In [ ]:
site_polder_intersection = (
    restoration_sites_buf.sjoin(
        protected_polders_9678, how="left", predicate="intersects"
    )[["potential_site_id", "polder_ID"]]
    .sort_values(by="polder_ID")
    .reset_index(drop=True)
    .dropna()
)

In [ ]:
sites_link_polders = geopandas.GeoDataFrame(
    site_polder_intersection.sort_values(by="potential_site_id")
    .groupby("potential_site_id")
    .agg(lambda d: sorted(d))
    .join(
        restoration_sites.set_index("potential_site_id").drop(columns=["mean", "class"])
    ),
    crs=restoration_sites.crs,
)
sites_link_polders.to_file(
    "../scratch/potential_sites_with_polders.gpkg", driver="GPKG", engine="pyogrio"
)

In [ ]:
# each site is within DISTANCE_WITHIN_M of a set of one or more polders
# that may be protected if the site is restored
# (site 1 may protect polders A and B, site 2 => polder C)
site_to_polders = defaultdict(set)
# this is all the different sets of polders that are within DISTANCE_WITHIN_M
# of any potential site
# (polders A and B or polder C)
polder_sets = set()
# each set of polders is within DISTANCE_WITHIN_M of a set of sites that
# may protect them
# (polders A and B may be protected by site 1, polder C => site 2)
polder_set_to_sites = defaultdict(set)

lookup = site_polder_intersection.set_index("potential_site_id")
for site in site_polder_intersection.potential_site_id.unique():
    site_polders = lookup.loc[site, "polder_ID"]
    if isinstance(site_polders, str):
        polder_set = frozenset([site_polders])
    else:
        polder_set = frozenset(site_polders)
    site_to_polders[site] = polder_set
    polder_sets.add(polder_set)
    polder_set_to_sites[polder_set].add(site)

In [ ]:
polder_site_relations = []
for polders, sites in polder_set_to_sites.items():
    polder_site_relations.append(
        {
            "polder_set": sorted(polders),
            "sites_protecting": sorted(int(s) for s in sites),
        }
    )
with open("../scratch/polder_site_relations.json", "w") as fh:
    json.dump(polder_site_relations, fh, indent=2)

## Feature polder intersection

To find potential benefit or assets-at-risk

In [ ]:
def read_feature_damages(
    base_path, hazard, sector, infrastructure_path, damage_path, polders
):
    # print("*", hazard, sector)
    fpath = base_path / infrastructure_path
    fpi_path = Path("../scratch") / f"{fpath.name}.pq"
    if not fpi_path.exists():
        features = (
            geopandas.read_file(
                base_path / infrastructure_path, engine="pyogrio", fid_as_index=True
            )
            .reset_index()
            .to_crs(epsg=4326)
        )
        # print("read features", len(features))
        feature_polder_intersection = features.sjoin(
            polders[["polder_ID", "geometry"]], how="inner", predicate="intersects"
        ).reset_index(drop=True)
        # print("intersected features", len(feature_polder_intersection))
        # print("unique fid", len(feature_polder_intersection.fid.unique()))
        feature_polder_intersection.to_parquet(fpi_path)
    else:
        feature_polder_intersection = geopandas.read_parquet(fpi_path)
        # print("read intersected features", len(feature_polder_intersection))

    dpath = base_path / damage_path
    dpi_path = Path("../scratch") / f"{dpath.name}.pq"
    if not dpi_path.exists():
        damage = pandas.read_csv(base_path / damage_path).set_index("fid")
        # print("read damage", len(damage))

        try:
            damage_polder_intersection = damage.loc[feature_polder_intersection.fid]
        except KeyError as e:
            # print("Warning: Ignoring invalid keys (features not present in calculated damages)")
            valid_fid = damage.index.intersection(feature_polder_intersection.fid)
            damage_polder_intersection = damage.loc[valid_fid]
        # print("polder damage", len(damage_polder_intersection))

        damage_columns = [
            c for c in damage_polder_intersection.columns if "Damage" in c
        ]

        df_damage = damage_polder_intersection[damage_columns]
        mask = df_damage.max(axis=1) > 0
        damage_polder_intersection_nonzero_damage = damage_polder_intersection[mask]
        # print("filtered damage", len(damage_polder_intersection_nonzero_damage))

        damage_polder_intersection_nonzero_damage.to_parquet(dpi_path)
    else:
        damage_polder_intersection_nonzero_damage = pandas.read_parquet(dpi_path)
        # print("read filtered damage", len(damage_polder_intersection_nonzero_damage))

    return feature_polder_intersection, damage_polder_intersection_nonzero_damage


metadata = pandas.read_csv("infra-damage.csv")
summary_dfs = []
for row in metadata.itertuples():
    _, hazard, sector, infrastructure_path, damage_path = row
    print(hazard, sector)
    feature_polder_intersection, damage_polder_intersection = read_feature_damages(
        base_path,
        hazard,
        sector,
        infrastructure_path,
        damage_path,
        protected_polders_4326,
    )
    feature_polder_intersection[
        "length_m"
    ] = feature_polder_intersection.geometry.to_crs(epsg=9678).length

    features = feature_polder_intersection[["fid", "polder_ID", "length_m"]].set_index(
        "fid"
    )
    damage_columns = [c for c in damage_polder_intersection.columns if "Damage" in c]
    damage = damage_polder_intersection[damage_columns]

    df = (
        features.join(damage)
        .reset_index()
        .melt(
            id_vars=["fid", "polder_ID", "length_m"],
            var_name="scenario",
            value_name="damage_euro",
        )
    )
    meta = df.scenario.str.extract(
        r"Damage_euro_(?P<epoch>\w\w\w\w)_?(?P<rcp>\w+)?_rp(?P<rp>\d+)"
    )
    df = pandas.concat([df.drop(columns=["scenario"]), meta], axis=1)
    df.rcp = df.rcp.fillna("baseline")
    df.rp = df.rp.astype(int)
    df["hazard"] = hazard
    df["sector"] = sector
    df["count"] = 1
    df["count_damaged"] = df.damage_euro > 0
    df["damaged_length_m"] = df.length_m * df.count_damaged
    summary = df.groupby(
        by=["polder_ID", "hazard", "sector", "epoch", "rcp", "rp"]
    ).agg(
        {
            "count": "sum",
            "count_damaged": "sum",
            "length_m": "sum",
            "damaged_length_m": "sum",
            "damage_euro": "sum",
        }
    )
    summary_dfs.append(summary)

summary = pandas.concat(summary_dfs)
summary

In [ ]:
summary.to_csv(
    "../scratch/polder_rp_damages.csv", float_format="%.2f"
)  # output rounded to the cent !!!